In [ ]:
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.under_sampling import RandomUnderSampler
#from google.colab import drive

In [ ]:
#drive.mount("/content/drive/") #apenas para o google collab

Mounted at /content/drive/


In [ ]:
def get_train_data(n):
    df = pd.read_csv("/content/drive/Shareddrives/Mineração de Dados/data/all_data.csv")
    df = df[df.split == "train"]
    df = df.dropna(subset=['comment_text', 'toxicity'])
  
    X = df.comment_text.to_numpy().reshape(-1,1)
    y = df.toxicity >= 0.5

    under_sampler = RandomUnderSampler(random_state=0)
    X, y = under_sampler.fit_resample(X, y)
    df = pd.DataFrame({"comment_text": X.flatten(), "toxicity": y})
    df = shuffle(df, random_state = 0)
    df = df.sample(n=n, random_state=42)
    return df

def get_test_data(n):
    df = pd.read_csv("/content/drive/Shareddrives/Mineração de Dados/data/all_data.csv")
    df = df[df.split == "test"]
    df = df.dropna(subset=['comment_text', 'toxicity'])
  
    X = df.comment_text.to_numpy().reshape(-1,1)
    y = df.toxicity >= 0.5

    under_sampler = RandomUnderSampler(random_state=42)
    X, y = under_sampler.fit_resample(X, y)
    df = pd.DataFrame({"comment_text": X.flatten(), "toxicity": y})
    df = shuffle(df, random_state = 42)
    df = df.sample(n=n, random_state=42)
    return df

# CNN

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/Mineração de Dados/data/all_data.csv")

In [ ]:
df.dropna(subset=['comment_text', 'toxicity'], inplace=True)
df.dropna(axis='columns', inplace=True)

In [ ]:
df.drop_duplicates(subset=['comment_text',], keep='first', inplace=True)

In [ ]:
df.comment_text = df.comment_text.replace('\xad', '', regex=True) 

In [ ]:
#!pip install contractions
import contractions
df.comment_text = df.comment_text.apply(contractions.fix)

     |████████████████████████████████| 284 kB 4.0 MB/s 
     |████████████████████████████████| 321 kB 66.8 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85452 sha256=904200708897ce24ce62bc26d98af4d955c41e2ca70b378b8c6be347fee18e79
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
df['label'] = np.where(df['toxicity'] >= 0.5, 1, 0)

In [ ]:
#!pip install datasets
from datasets import Dataset, DatasetDict
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


X = df.comment_text.to_numpy().reshape(-1,1)
y = np.where(df[['toxicity']] >= 0.5, 1, 0).reshape(-1,1)

under_sampler = RandomUnderSampler(random_state=0)
X, y = under_sampler.fit_resample(X, y)

raw_datasets = Dataset.from_dict({
    'comment_text': X.ravel(),
    'label': y,
}).train_test_split(train_size=0.9, test_size=0.1)

raw_datasets


     |████████████████████████████████| 306 kB 4.6 MB/s 
     |████████████████████████████████| 243 kB 63.5 MB/s 
     |████████████████████████████████| 133 kB 65.8 MB/s 
     |████████████████████████████████| 1.1 MB 58.4 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 271 kB 63.5 MB/s 
     |████████████████████████████████| 160 kB 77.5 MB/s 
     |████████████████████████████████| 192 kB 80.7 MB/s 


DatasetDict({
    train: Dataset({
        features: ['comment_text', 'label'],
        num_rows: 283813
    })
    test: Dataset({
        features: ['comment_text', 'label'],
        num_rows: 31535
    })
})

In [ ]:
def create_model(max_n_words, len_embedding, input_length):
    model = tf.keras.Sequential()
    model.add(layers.Embedding(max_n_words, len_embedding, input_length=input_length))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv1D(64, 3, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dropout(0.2))  
    #model.add(layers.Bidirectional(layers.LSTM(100)))
    model.add(layers.Dense(2, activation='sigmoid'))
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss="binary_crossentropy",
                  optmizer=opt,
                  #optimizer=tf.optimizers.SGD(learning_rate=0.01),
                  metrics=["accuracy"])
    return model

In [ ]:
max_seq_len = 300
len_embedding = 300

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.pipeline import Pipeline

nltk.download('stopwords')

tknzr = TweetTokenizer(preserve_case=False, reduce_len=False)
vectorizer = TfidfVectorizer(ngram_range=(1,1), tokenizer=tknzr.tokenize, strip_accents='unicode', lowercase=True, stop_words=stopwords.words('english'))

pipeline = Pipeline([
    ('vectorizer', vectorizer),
])
features = pipeline.fit_transform(raw_datasets['train']['comment_text'], raw_datasets['train']['label'])
feature_names = pipeline['vectorizer'].get_feature_names_out()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=len(feature_names), filters='')
tokenizer.fit_on_texts(raw_datasets["train"]["comment_text"])

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 537105 unique tokens.


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_cnn = tokenizer.texts_to_sequences(raw_datasets["train"]["comment_text"])
X_cnn = pad_sequences(X_cnn, maxlen=max_seq_len)

In [ ]:
max_n_words = len(feature_names)
max_n_words

159630

In [ ]:
model = create_model(max_n_words, len_embedding, X_cnn.shape[1])

In [ ]:
y = pd.get_dummies(raw_datasets["train"]["label"]).values
y

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [1, 0],
       [1, 0]], dtype=uint8)

In [ ]:
X_cnn

array([[    0,     0,     0, ...,    41,    28,  3347],
       [    0,     0,     0, ..., 46117, 23809,  4998],
       [    0,     0,     0, ...,     4,    45,  6883],
       ...,
       [    0,     0,     0, ...,    58,    17, 44226],
       [    0,     0,     0, ...,     5,   428, 15681],
       [    0,     0,     0, ...,    34,    15, 28701]], dtype=int32)

In [ ]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [ ]:
model.fit(X_cnn, y, epochs=10, batch_size=64, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.01)])

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


3992/3992 [==============================] - 194s 46ms/step - loss: 0.4474 - accuracy: 0.8096 - val_loss: 0.4531 - val_accuracy: 0.7919
Epoch 2/10
3992/3992 [==============================] - 186s 47ms/step - loss: 0.4058 - accuracy: 0.8421 - val_loss: 0.4047 - val_accuracy: 0.8362
Epoch 3/10
3992/3992 [==============================] - 184s 46ms/step - loss: 0.3818 - accuracy: 0.8591 - val_loss: 0.4268 - val_accuracy: 0.8362
Epoch 4/10
3992/3992 [==============================] - 183s 46ms/step - loss: 0.3503 - accuracy: 0.8753 - val_loss: 0.5364 - val_accuracy: 0.8274
Epoch 5/10
3992/3992 [==============================] - 184s 46ms/step - loss: 0.3311 - accuracy: 0.8866 - val_loss: 0.5113 - val_accuracy: 0.8270


In [ ]:
y_test = pd.get_dummies(raw_datasets["test"]["label"]).values
y_test

array([[0, 1],
       [1, 0],
       [0, 1],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [ ]:
X_test = tokenizer.texts_to_sequences(raw_datasets["test"]["comment_text"])
X_test = pad_sequences(X_test, maxlen=max_seq_len)

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_cnn = np.argmax(y_pred, axis=1)

print(classification_report(np.array(raw_datasets["test"]["label"]), y_pred_cnn))

 23/493 [>.............................] - ETA: 2s

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


493/493 [==============================] - 2s 4ms/step
              precision    recall  f1-score   support

           0       0.79      0.88      0.83     15687
           1       0.86      0.77      0.81     15848

    accuracy                           0.82     31535
   macro avg       0.83      0.82      0.82     31535
weighted avg       0.83      0.82      0.82     31535



In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

print("F1:", f1_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn))
print("Recall:", recall_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn))
print("Precisão:", precision_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn))
print("Acurácia:", accuracy_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn))

F1: 0.8149135374670975
Recall: 0.7716431095406361
Precisão: 0.8633250970702435
Acurácia: 0.8238465197399715


In [ ]:
print("Precisão:", precision_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn, average=None))
print("Recall:", recall_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn, average=None))
print("F1:", f1_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn, average=None))

Precisão: [0.79165227 0.8633251 ]
Recall: [0.87658571 0.77164311]
F1: [0.83195692 0.81491354]


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 300, 300)          47889000  
                                                                 
 dropout_2 (Dropout)         (None, 300, 300)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 298, 64)           57664     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                      

# LSTM


In [ ]:
def create_lstm(max_n_words, len_embedding, input_length):
    model = tf.keras.Sequential()
    model.add(layers.Embedding(max_n_words, len_embedding, input_length=input_length))
    model.add(layers.LSTM(100, dropout=0.2))
    #model.add(layers.Bidirectional(layers.LSTM(100)))
    model.add(layers.Dense(2, activation='sigmoid'))
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  #optimizer=tf.optimizers.SGD(learning_rate=0.01),
                  metrics=["accuracy"])
    return model

In [ ]:
lstm = create_lstm(max_n_words, len_embedding, X_cnn.shape[1])

In [ ]:
y_train = pd.get_dummies(raw_datasets["train"]["label"]).values

In [ ]:
lstm.fit(X_cnn, y_train, epochs=10, batch_size=64, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.01)])

Epoch 1/10
   2/3992 [..............................] - ETA: 4:56 - loss: 0.6931 - accuracy: 0.4531

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


3992/3992 [==============================] - 247s 62ms/step - loss: 0.4013 - accuracy: 0.8243 - val_loss: 0.3695 - val_accuracy: 0.8432
Epoch 2/10
3992/3992 [==============================] - 253s 63ms/step - loss: 0.3345 - accuracy: 0.8618 - val_loss: 0.3849 - val_accuracy: 0.8375
Epoch 3/10
3992/3992 [==============================] - 248s 62ms/step - loss: 0.3144 - accuracy: 0.8715 - val_loss: 0.3871 - val_accuracy: 0.8399
Epoch 4/10
3992/3992 [==============================] - 248s 62ms/step - loss: 0.2976 - accuracy: 0.8794 - val_loss: 0.3917 - val_accuracy: 0.8370


In [ ]:
y_pred = lstm.predict(X_test, batch_size=64, verbose=1)
y_pred_lstm = np.argmax(y_pred, axis=1)

print(classification_report(np.array(raw_datasets["test"]["label"]), y_pred_lstm))

  9/493 [..............................] - ETA: 7s

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


493/493 [==============================] - 6s 12ms/step
              precision    recall  f1-score   support

           0       0.83      0.85      0.84     15687
           1       0.85      0.83      0.84     15848

    accuracy                           0.84     31535
   macro avg       0.84      0.84      0.84     31535
weighted avg       0.84      0.84      0.84     31535



In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

print("F1:", f1_score(np.array(raw_datasets["test"]["label"]), y_pred_lstm))
print("Recall:", recall_score(np.array(raw_datasets["test"]["label"]), y_pred_lstm))
print("Precisão:", precision_score(np.array(raw_datasets["test"]["label"]), y_pred_lstm))
print("Acurácia:", accuracy_score(np.array(raw_datasets["test"]["label"]), y_pred_lstm))

F1: 0.8386953204476093
Recall: 0.8323447753659767
Precisão: 0.8451435161455664
Acurácia: 0.8390994133502457


In [ ]:
print("Precisão:", precision_score(np.array(raw_datasets["test"]["label"]), y_pred_lstm, average=None))
print("Recall:", recall_score(np.array(raw_datasets["test"]["label"]), y_pred_lstm, average=None))
print("F1:", f1_score(np.array(raw_datasets["test"]["label"]), y_pred_lstm, average=None))

Precisão: [0.83317637 0.84514352]
Recall: [0.84592338 0.83234478]
F1: [0.83950149 0.83869532]


In [ ]:
lstm.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 300, 300)          47889000  
                                                                 
 lstm_4 (LSTM)               (None, 100)               160400    
                                                                 
 dense_6 (Dense)             (None, 2)                 202       
                                                                 
Total params: 48,049,602
Trainable params: 48,049,602
Non-trainable params: 0
_________________________________________________________________


# CNN+LSTM

In [ ]:
def create_lstm_cnn(max_n_words, len_embedding, input_length):
    model = tf.keras.Sequential()
    model.add(layers.Embedding(max_n_words, len_embedding, input_length=input_length))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv1D(64, 3, activation='relu'))
    model.add(layers.MaxPooling1D())
    model.add(layers.LSTM(100, dropout=0.2))
    model.add(layers.Dense(2, activation='sigmoid'))
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  #optimizer=tf.optimizers.SGD(learning_rate=0.01),
                  metrics=["accuracy"])
    return model

In [ ]:
cnn_lstm = create_lstm(max_n_words, len_embedding, X_cnn.shape[1])

In [ ]:
cnn_lstm.fit(X_cnn, y_train, epochs=10, batch_size=64, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.01)])

Epoch 1/10
   1/3992 [..............................] - ETA: 5:31 - loss: 0.6931 - accuracy: 0.4844

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


3992/3992 [==============================] - 247s 62ms/step - loss: 0.4016 - accuracy: 0.8229 - val_loss: 0.3789 - val_accuracy: 0.8407
Epoch 2/10
3992/3992 [==============================] - 245s 61ms/step - loss: 0.3446 - accuracy: 0.8565 - val_loss: 0.4018 - val_accuracy: 0.8300
Epoch 3/10
3992/3992 [==============================] - 248s 62ms/step - loss: 0.3327 - accuracy: 0.8632 - val_loss: 0.4160 - val_accuracy: 0.8247
Epoch 4/10
3992/3992 [==============================] - 244s 61ms/step - loss: 0.3270 - accuracy: 0.8656 - val_loss: 0.4183 - val_accuracy: 0.8262


In [ ]:
y_pred = cnn_lstm.predict(X_test, batch_size=64, verbose=1)
y_pred_cnn_lstm = np.argmax(y_pred, axis=1)

print(classification_report(np.array(raw_datasets["test"]["label"]), y_pred_cnn_lstm))

  9/493 [..............................] - ETA: 7s

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


493/493 [==============================] - 6s 12ms/step
              precision    recall  f1-score   support

           0       0.83      0.83      0.83     15687
           1       0.83      0.83      0.83     15848

    accuracy                           0.83     31535
   macro avg       0.83      0.83      0.83     31535
weighted avg       0.83      0.83      0.83     31535



In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

print("F1:", f1_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn_lstm))
print("Recall:", recall_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn_lstm))
print("Precisão:", precision_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn_lstm))
print("Acurácia:", accuracy_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn_lstm))

F1: 0.8307808421648479
Recall: 0.8334805653710248
Precisão: 0.8280985518149332
Acurácia: 0.8293641985095925


In [ ]:
print("Precisão:", precision_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn_lstm, average=None))
print("Recall:", recall_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn_lstm, average=None))
print("F1:", f1_score(np.array(raw_datasets["test"]["label"]), y_pred_cnn_lstm, average=None))


Precisão: [0.83065965 0.82809855]
Recall: [0.82520558 0.83348057]
F1: [0.82792364 0.83078084]


In [ ]:
cnn_lstm.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 300, 300)          47889000  
                                                                 
 lstm_5 (LSTM)               (None, 100)               160400    
                                                                 
 dense_7 (Dense)             (None, 2)                 202       
                                                                 
Total params: 48,049,602
Trainable params: 48,049,602
Non-trainable params: 0
_________________________________________________________________
